1. Change notebook name
2. Change df name (80 occurances)
3. Change json file name
4. Set collected on date


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [2]:
start_time = timeit.default_timer()


df_2019_nov = pd.read_json('json_datasets/Kickstarter_2019-11-14T03_20_27_004Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 133.056413602


In [3]:
# Save the creation datetime for calculating age of project later
df_2019_nov_collected_on = datetime(2019, 11, 14)

In [4]:
# clean up the format and make a deep copy
start_time = timeit.default_timer()


df_2019_nov = df_2019_nov['data'].apply(pd.Series)
df_2019_nov_copy = df_2019_nov.copy(deep=True)
print(f'Total_rows: {len(df_2019_nov)}')


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

df_2019_nov.head(3)

Total_rows: 211424
time: 110.65276263400006


,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,country_displayable_name,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,current_currency,usd_type,creator,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,117424629,{'key': 'assets/012/167/578/bc9a45402692d32490...,Bring the Thunder Gumbo Art Car to Burning Man...,Help support the legendary B.O.O.G.A. mutant v...,6000.0,6223.0,successful,thunder-gumbo-art-car-burning-man-2015-repairs...,False,US,the United States,USD,$,True,1438323004,1438323004,1434317481,1435731004,False,False,85,1.000000,6223.0,6223,1.000000,USD,international,"{'id': 1120701030, 'name': 'Shalin Scupham', '...","{'id': 99999999, 'name': 'Black Rock City', 's...","{'id': 53, 'name': 'Public Art', 'slug': 'art/...","{'id': 1958146, 'project_id': 1958146, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,699034184,{'key': 'assets/013/470/130/f533d172ac45032e2b...,Little D Threads (Canceled),Handmade Organic comfortable trendy clothes fo...,15000.0,0.0,canceled,little-d-threads,False,CA,Canada,CAD,$,True,1474582606,1472164799,1471976279,1471990606,False,False,0,0.774623,0.0,0,0.754185,USD,domestic,"{'id': 2039752660, 'name': 'Cassandra Derosa',...","{'id': 8676, 'name': 'Edmonton', 'slug': 'edmo...","{'id': 264, 'name': 'Childrenswear', 'slug': '...","{'id': 2649999, 'project_id': 2649999, 'state'...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,44517997,{'key': 'assets/011/826/861/e5a4a8c96a3bb0de9d...,Dawn of the Anthropocene - Public Art Ice Scul...,"In NYC, 2 days before the U.N. Climate Summit,...",4500.0,5667.0,successful,dawn-of-the-anthropocene-public-art-ice-sculpture,False,US,the United States,USD,$,True,1412170599,1412170599,1407432996,1408714599,True,False,98,1.000000,5667.0,5667,1.000000,USD,international,"{'id': 627800435, 'name': 'LigoranoReese', 'sl...","{'id': 2459115, 'name': 'New York', 'slug': 'n...","{'id': 53, 'name': 'Public Art', 'slug': 'art/...","{'id': 1270400, 'project_id': 1270400, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [5]:
# Check if data has same features
df_2019_nov_cols = df_2019_nov.columns
columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
diff = list(set(df_2019_nov_cols) - set(columns_original))
diff2 = list(set(columns_original) - set(df_2019_nov_cols))
print(diff)
print(diff2)

[]
[]


In [6]:
# inspect canceled and suspended states
df_2019_nov.state.value_counts()

successful    119652
failed         75100
canceled        8681
live            7350
suspended        641
Name: state, dtype: int64

In [7]:
# drop canceled and suspended states
df_2019_nov.drop((df_2019_nov[((df_2019_nov.state == 'suspended') | 
                                         (df_2019_nov.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2019_nov.duplicated(['id']).sum()}")

# drop duplicates
df_2019_nov.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2019_nov)}')


Rows with duplicate project_id: 26889
Total_rows after dropping duplicates and suspended/canceled states: 175213


In [ ]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2019_nov['project_link'] = 'no_val'
# project_link_loc = df_2019_nov.columns.get_loc('project_link')
# for n in range(len(df_2019_nov)):
#     project_link = df_2019_nov.iloc[n].urls['web']['project']
#     df_2019_nov.iat[n, project_link_loc] = project_link
    
# df_2019_nov['creator_link'] = 'no_val'
# creator_link_loc = df_2019_nov.columns.get_loc('creator_link')
# for n in range(len(df_2019_nov)):
#     creator_link = df_2019_nov.iloc[n].creator['urls']['web']['user']
#     df_2019_nov.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [ ]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2019_nov['main_category'] = 'no_value'
# cat_loc = df_2019_nov.columns.get_loc('main_category')


# for n in (range(len(df_2019_nov))):
#     if 'parent_id' not in df_2019_nov.iloc[n]['category'].keys():
#         cat_id = df_2019_nov.iloc[n]['category']['id']
#         df_2019_nov.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2019_nov.iloc[n]['category']['parent_id']):
#         cat_id = df_2019_nov.iloc[n]['category']['id']
#         df_2019_nov.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2019_nov.iloc[n]['category']['parent_id']
#         df_2019_nov.iat[n, cat_loc] = par_id
        
# df_2019_nov['main_category'] = df_2019_nov['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2019_nov.head(1)

In [8]:
# keep these features
df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
'backers_count', 'static_usd_rate', 'usd_pledged' ]
# Drop these 
df_drop_feats = list(set(df_2019_nov_cols) - set(df_main_feats))
print(f'Columns removed: {df_drop_feats}')

# drop features
df_2019_nov.drop(df_drop_feats, axis=1, inplace=True)

Columns removed: ['creator', 'converted_pledged_amount', 'current_currency', 'fx_rate', 'slug', 'usd_type', 'category', 'created_at', 'currency_trailing_code', 'friends', 'photo', 'disable_communication', 'urls', 'permissions', 'location', 'currency_symbol', 'profile', 'is_backing', 'country_displayable_name', 'state_changed_at', 'is_starred', 'is_starrable', 'spotlight', 'source_url']


In [9]:
# convert unix dates to datetime objects
df_2019_nov.deadline = df_2019_nov.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2019_nov.launched_at = df_2019_nov.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [10]:
# rename columns
df_2019_nov.rename(columns={'id':'project_id'}, inplace=True)
df_2019_nov.rename(columns={'name':'project_name'}, inplace=True)
df_2019_nov.rename(columns={'blurb':'project_description'}, inplace=True)
df_2019_nov.rename(columns={'state':'outcome_state'}, inplace=True)


In [11]:
# add project duration feature
df_2019_nov['duration'] = df_2019_nov.deadline - df_2019_nov.launched_at
df_2019_nov['days_from_launch'] =(df_2019_nov_collected_on - df_2019_nov.launched_at).apply(lambda x: x.days)

# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_nov['duration_mid_point'] = df_2019_nov.duration.apply(lambda x: x.days) / 2
df_2019_nov['lower_bound_mid_point_range'] = df_2019_nov.duration_mid_point.apply(lambda x: x - 5)
df_2019_nov['upper_bound_mid_point_range'] = df_2019_nov.duration_mid_point.apply(lambda x: x + 5)
df_2019_nov['in_mid_duration_range'] = df_2019_nov.days_from_launch.between(df_2019_nov.lower_bound_mid_point_range, df_2019_nov.upper_bound_mid_point_range)

df_2019_nov.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [20]:
# df_2019_nov['duration'][0].quantile(.1)
df_2019_nov['duration_10_percent'] = df_2019_nov.duration.apply(lambda x: x.days / 10 ) 


In [21]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_nov['lower_bound'] = df_2019_nov.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2019_nov['upper_bound'] = df_2019_nov.duration_10_percent.apply(lambda x: x + 5)
df_2019_nov['in_range'] = df_2019_nov.days_from_launch.between(df_2019_nov.lower_bound, df_2019_nov.upper_bound)

# df_2019_nov.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [22]:
# calculate usd_goal
df_2019_nov['usd_goal'] = df_2019_nov.goal * df_2019_nov.static_usd_rate


In [23]:
# Replace goal and rearrange
col_list = df_2019_nov.columns.to_list()
print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2019_nov = df_2019_nov[col_list_rearrange]
df_2019_nov.head(1)
                      

['project_id', 'project_name', 'project_description', 'goal', 'pledged', 'outcome_state', 'country', 'currency', 'deadline', 'launched_at', 'staff_pick', 'backers_count', 'static_usd_rate', 'usd_pledged', 'duration', 'days_from_launch', 'in_mid_duration_range', 'duration_10_percent', 'lower_bound', 'upper_bound', 'in_range', 'usd_goal']


,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
0,117424629,Bring the Thunder Gumbo Art Car to Burning Man...,Help support the legendary B.O.O.G.A. mutant v...,successful,False,3.0,1596,6000.0,6223.0,USD,2015-07-01 02:10:04,2015-07-31 02:10:04,False,US,30 days,85


In [26]:
df_2019_nov_copy = df_2019_nov.copy(deep=True)
# convert datetime columns before exporting
# df_2019_nov.duration = df_2019_nov.duration.apply(lambda x: x.days) 
df_2019_nov.launched_at = df_2019_nov.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2019_nov.deadline = df_2019_nov.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

AttributeError: 'str' object has no attribute 'strftime'

In [29]:
# export cleaned dataframe to csv
df_2019_nov.to_csv('clean_datasets_10/main_df_2019_nov.csv', index=False)